In [7]:
import json
import argparse
import glob
import sys
sys.path.append("../")
from tool import finqa_equal
from typing import Dict, Any


In [8]:
name = 'finqa_gpt-35-turbo_04_10_14_54.jsonl'

In [9]:
exception = []
correct, wrong, missing = 0, 0, 0
with open(name) as f:
    for i, line in enumerate(f):
        data = json.loads(line)
        if 'demonstration' in data or 'prompt' in data:
            continue
        else:
            # Select the prediction field.
            if 'executed' in data:
                prediction = data['executed']
            else:
                prediction = data['prediction']
                if False:
                    # For multi-choise question, pick A option
                    if prediction == 'None':
                        prediction = 'A'
            # Calculate the evaluation score
            pred = finqa_equal(prediction, data['answer'], True, True)
            if prediction is None:
                missing += 1
            elif pred:
                correct += 1
            else:
                exception.append(data)
                wrong += 1

print('file name: ', name)
print('num of examples: ', correct + wrong + missing)
print('num of missing returns', missing)
print('accuracy: ', correct / (correct + wrong + missing))

file name:  finqa_gpt-35-turbo_04_10_14_54.jsonl
num of examples:  1147
num of missing returns 0
accuracy:  0.6120313862249346


In [10]:
import pandas as pd

In [11]:
incorrect = pd.DataFrame(exception)

In [12]:
finqa_prompt = [{'role': 'system',
  'content': 'You are an AI assistant that helps people find information.'},
 {'role': 'user',
  'content': '''Read the following text and table, and then write code to answer a question:
( in millions ) | dec 282013 | dec 292012
available-for-sale investments | $ 18086 | $ 14001
cash | 854 | 593
equity method investments | 1038 | 992
loans receivable | 1072 | 979
non-marketable cost method investments | 1270 | 1202
reverse repurchase agreements | 800 | 2850
trading assets | 8441 | 5685
total cash and investments | $ 31561 | $ 26302
Question: what percentage of total cash and investments as of dec . 29 2012 was comprised of available-for-sale investments?'''},
 {'role': 'assistant',
  'content': '''#Python
available_for_sale_investments_dec_29_2012 = 14001
total_cash_and_investments_dec_29_2012 = 26302
percent_available_for_sale_investments_dec_29_2012 = available_for_sale_investments_dec_29_2012 / total_cash_and_investments_dec_29_2012
ans = percent_available_for_sale_investments_dec_29_2012'''},

{'role': 'user',
  'content': '''Read the following text and table, and then write code to answer a question:
the chart shows that the firm posted market risk 2013related gains on 248 out of 261 days in this period , with 12 days exceeding $ 210 million .
december 31 ( in millions ) | 1 basis point increase in jpmorgan chase 2019s credit spread
2010 | $ 35
2009 | $ 39
Question: on what percent of trading days were there market gains above $ 210 million?'''},
{'role': 'assistant',
  'content': '''#Python
days_with_market_gains_above_210_million = 12
total_trading_days = 261
percent_days_with_market_gains_above_210_million = days_with_market_gains_above_210_million / total_trading_days
ans = percent_days_with_market_gains_above_210_million'''},

{'role': 'user',
  'content': '''Read the following text and table, and then write code to answer a question:
american tower corporation and subsidiaries notes to consolidated financial statements ( 3 ) consists of customer-related intangibles of approximately $ 75.0 million and network location intangibles of approximately $ 72.7 million . the customer-related intangibles and network location intangibles are being amortized on a straight-line basis over periods of up to 20 years .
- | preliminary purchase price allocation
current assets | $ 8763
non-current assets | 2332
property and equipment | 26711
intangible assets ( 1 ) | 21079
other non-current liabilities | -1349 ( 1349 )
fair value of net assets acquired | $ 57536
goodwill ( 2 ) | 5998
Question: for acquired customer-related and network location intangibles , what is the expected annual amortization expenses , in millions?'''},
{'role': 'assistant',
  'content': '''#Python
customer_related_intangibles = 75
network_location_intangibles = 72.7
straight_line_basis = 20
amortization_expenses = ( customer_related_intangibles + network_location_intangibles ) / straight_line_basis
ans = amortization_expenses'''},

{'role': 'user',
  'content': '''Read the following text and table, and then write code to answer a question:
the aggregate commitment under the liquidity asset purchase agreements was approximately $ 23.59 billion and $ 28.37 billion at december 31 , 2008 and 2007 , respectively .
( dollars in billions ) | 2008 amount | 2008 percent of total conduit assets | 2008 amount | percent of total conduit assets
united states | $ 11.09 | 46% ( 46 % ) | $ 12.14 | 42% ( 42 % )
australia | 4.30 | 17 | 6.10 | 21
great britain | 1.97 | 8 | 2.93 | 10
spain | 1.71 | 7 | 1.90 | 7
italy | 1.66 | 7 | 1.86 | 7
portugal | 0.62 | 3 | 0.70 | 2
germany | 0.57 | 3 | 0.70 | 2
netherlands | 0.40 | 2 | 0.55 | 2
belgium | 0.29 | 1 | 0.31 | 1
greece | 0.27 | 1 | 0.31 | 1
other | 1.01 | 5 | 1.26 | 5
total conduit assets | $ 23.89 | 100% ( 100 % ) | $ 28.76 | 100% ( 100 % )
Question: what is percentage change in total conduit asset from 2007 to 2008?'''},
{'role': 'assistant',
  'content': '''#Python
total_conduit_assets_2007 = 28.76
total_conduit_assets_2008 = 23.89
net_change_in_total_conduit_assets = total_conduit_assets_2008 - total_conduit_assets_2007
percent_change_in_total_conduit_assets = net_change_in_total_conduit_assets / total_conduit_assets_2007
ans = percent_change_in_total_conduit_assets'''},]

In [13]:
def create_reader_request_processed(example: Dict[str, Any]):
    prompt = 'Read the following text and table, and then write code to answer a question:\n'
    if example['text']:
        prompt += example['text'] + '\n'
    prompt += example['table'].strip() + '\n'
    prompt += 'Question: {}\n'.format(example['question'])
    return prompt

In [14]:
full_prompt = prompt_4shot + "\n\n"+ create_reader_request_processed(incorrect.iloc[1,])

In [15]:
incorrect['prompt'] = incorrect.apply(lambda x:  prompt_4shot + "\n\n"+ create_reader_request_processed(x), axis=1)


In [16]:
incorrect.columns

Index(['question', 'answer', 'text', 'table', 'id', 'program', 'generated',
       'executed', 'prompt'],
      dtype='object')

In [17]:
incorrect

,question,answer,text,table,id,program,generated,executed,prompt
0,what is the percentage change in cash flow hed...,0.09864,undesignated hedges was $ 41.2 million and $ 4...,statement of income classification | statement...,ADI/2011/page_61.pdf-2,x0 = 153.7 - 139.9; ans = x0 / 139.9,[There is no information provided in the text ...,There is no information provided in the text a...,"Read the following text and table, and then an..."
1,what was the percentage change in total rental...,0.06757,"total debt total debt at july 1 , 2006 was $ 1...",- | amount\n2007 | $ 56499000\n2008 | 46899000...,SYY/2006/page_71.pdf-1,x0 = 92710000 - 86842000; ans = x0 / 86842000,[#Python\nrental_expense_operating_leases_july...,0.159462,"Read the following text and table, and then an..."
2,did altria outperform the s&p 500?,yes,2011 2012 2013 2014 2015 2016 comparison of fi...,date | altria group inc . | altria group inc ....,MO/2016/page_19.pdf-1,ans = 286.61 > 198.09,[#Python\naltria_group_inc_return_2011 = 100\n...,1.0,"Read the following text and table, and then an..."
3,what portion of the total noncancelable future...,0.24566,some operating leases require payment of prope...,in millions | operating leases | capital lease...,GIS/2018/page_110.pdf-2,ans = 137.4 / 559.3,[#Python\noperating_leases_fiscal_2019 = 137.4...,0.246201,"Read the following text and table, and then an..."
4,was the five year total return of the 2019 pee...,no,2 0 1 9 a n n u a l r e p o r t1 6 performance...,- | 2014 | 2015 | 2016 | 2017 | 2018 | 2019\nj...,JKHY/2019/page_18.pdf-3,ans = 281.09 > 286.22,[#Python\ntotal_return_2019_peer_group_2014 = ...,0.0,"Read the following text and table, and then an..."
...,...,...,...,...,...,...,...,...,...
440,based on the review of the reconciliation of u...,-530.0,comcast corporation changes in our net deferre...,( in millions ) | 2015 | 2014 | 2013\nbalance ...,CMCSA/2015/page_112.pdf-3,ans = 1171 - 1701,[#Python\nreductions_for_tax_positions_of_prio...,-79.0,"Read the following text and table, and then an..."
441,what is the growth rate in the weighted averag...,0.16498,and subsidiaries notes to consolidated financi...,- | 2005 | 2006 | 2007\nrisk-free interest rat...,ABMD/2007/page_78.pdf-4,x0 = 8.05 - 6.91; ans = x0 / 6.91,[#Python\nweighted_average_fair_value_2005 = 8...,-0.141615,"Read the following text and table, and then an..."
442,what portion of the total long-term borrowings...,0.4386,borrowings short-term borrowings the carrying ...,( dollar amounts in millions ) | maturity amou...,BLK/2012/page_145.pdf-3,x0 = 750 + 1000; x1 = x0 + 750; ans = x1 / 5700,[#Python\ntotal_long_term_borrowings_due_in_ne...,0.439599,"Read the following text and table, and then an..."
443,what is the total expected contingent payments...,11.16667,and subsidiaries notes to consolidated financi...,balance at april 1 2007 | $ 224\nreductions fo...,ABMD/2008/page_86.pdf-4,x0 = 5583333 + 5583333; ans = x0 / 1000000,[There is no specific information provided in ...,There is no specific information provided in t...,"Read the following text and table, and then an..."


In [19]:
print(incorrect['prompt'][1])


Read the following text and table, and then answer the last question in a series of questions:
- | shares available for awards | shares subject to outstanding awards
2009 global incentive plan | 2322450 | 2530454
2004 stock incentive plan | - | 5923147
Questions: how many shares are subject to outstanding awards is under the 2009 global incentive plan? what about under the 2004 stock incentive plan? how many total shares are subject to outstanding awards? what about under the 2004 stock incentive plan?
Question: what proportion does this represent?
#Python
shares_subject_to_outstanding_awards_2009_global_incentive_plan = 2530454
shares_subject_to_outstanding_awards_2004_stock_incentive_plan = 5923147
total_shares_subject_to_outstanding_awards = shares_subject_to_outstanding_awards_2009_global_incentive_plan + shares_subject_to_outstanding_awards_2004_stock_incentive_plan
proportion = shares_subject_to_outstanding_awards_2009_global_incentive_plan / total_shares_subject_to_outstanding_a

In [148]:
i=100
print(incorrect['text'][i])
print(incorrect['questions'][i])
print('--------')
print(incorrect['generated'][i][0])
print('--------')
print(incorrect['golden_text'][i])
print('--------')
print(incorrect['golden_table'][i])
print('--------')
print(incorrect['programs'][i])
print(incorrect['answer'][i])


accumulated other comprehensive losses : pmi's accumulated other comprehensive losses , net of taxes , consisted of the following: . reclassifications from other comprehensive earnings the movements in accumulated other comprehensive losses and the related tax impact , for each of the components above , that are due to current period activity and reclassifications to the income statement are shown on the consolidated statements of comprehensive earnings for the years ended december 31 , 2015 , 2014 , and 2013 . the movement in currency translation adjustments for the year ended december 31 , 2013 , was also impacted by the purchase of the remaining shares of the mexican tobacco business . in addition , $ 1 million , $ 5 million and $ 12 million of net currency translation adjustment gains were transferred from other comprehensive earnings to marketing , administration and research costs in the consolidated statements of earnings for the years ended december 31 , 2015 , 2014 and 2013 , 

In [154]:

#Python
accumulated_other_comprehensive_losses_2013 = -4190
accumulated_other_comprehensive_losses_2014 = -6826
net_change_in_value = accumulated_other_comprehensive_losses_2014 - accumulated_other_comprehensive_losses_2013
ans = net_change_in_value
ans

-2636

In [120]:
print(incorrect['golden_text'][3])

total expense for repairs and maintenance incurred was $ 2.2 billion for 2011 , $ 2.0 billion for 2010 , and $ 1.9 billion for 2009 . 


In [115]:
print(incorrect['programs'][3])

['ans = 2.2 * 1000', 'ans = 325', 'x0 = 2.2 * 1000; ans = 325 + x0']
